# 2. Data cleaning and initial exploration

In this notebook we will be loading and performing an initial exploration of the dataset from the Protocols track for the Hercules challenge. This dataset consists of a series of 100 experimental protocols from the [Bio-Protocols journal](https://bio-protocol.org/Default.aspx).

## Setup
As always, we will begin by loading the logging system and setting up the path to import additional code from the _src_ folder.

In [1]:
%run __init__.py

INFO:root:Starting logger


In [2]:
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [3]:
from herc_common import BokehHistogram

hist = BokehHistogram(color_fill="mediumslateblue", color_hover="slateblue")

In [4]:
def print_empty_cols(df):
    for col in df.columns:
        print(col)
        print('-' * len(col))
        res = df[df[col] == ''].index
        print(f"{len(res)} articles have no value for column {col}")
        print(res)
        print('\n')


## Parsing the data
Since all the _html_ files of each protocol have been scraped in the previous phase, we will begin by getting the path of all the files that will be parsed later on:

In [5]:
import glob

files_to_read = glob.glob(f"{PROTOCOLS_DIR}/*.html")
len(files_to_read)

100

We will also define a simple Protocol class that will be used to stored the parsed information from every HTML file, and provides some utility methods to be loaded in a pandas DataFrame:

In [6]:
from src.protocol import Protocol

Since not every protocol will have a value for every defined field (for example, some protocols may not have a background section), we will also define a decorator that will be used later on to specify which fields are optional. These fields will be an empty string if no value can be found for them:

In [7]:
from src.data_reader import parse_protocol

protocols = []
for file in files_to_read:
    protocol_id = os.path.basename(file).split('.')[0]
    with open(file, 'r', encoding='utf-8') as f:
        protocols.append(parse_protocol(f.read(), protocol_id))

## Creating a dataframe

Now that every protocol has been parsed, we can load them into a pandas DataFrame:

In [8]:
import pandas as pd

df = pd.DataFrame([protocol.to_dict() for protocol in protocols])
df.head()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors
0,e100,Scratch Wound Healing Assay,The scratch wound healing assay has been widel...,"Human MDA-MB-231 cell line (ATCC, catalog numb...",Grow cells in DMEM supplemented with 10% FBS.|...,BD Falcon 24-well tissue culture plate (Fisher...,,Cancer Biology|Invasion & metastasis|Cell biol...,Yanling Chen
1,e1029,ADCC Assay Protocol,Antibody-dependent cell-mediated cytotoxicity ...,Raji cells (ATCC)|A/California/04/2009 (H1N1) ...,Preperation of Target Cells\n\n\t\t\n\n\n\t\t\...,Round bottomed 96-well plate|Temperature contr...,,Immunology|Immune cell function|Cytotoxicity|C...,Vikram Srivastava|Zheng Yang|Ivan Fan Ngai...
2,e1072,Catalase Activity Assay in Candida glabrata,Commensal and pathogenic fungi are exposed to ...,Yeast strains \nNote: BG14 was used as the C. ...,Preparation of total soluble extracts\n\t\t\n\...,Orbital incubator shaker|Microfuge tubes|50 ml...,,Microbiology|Microbial biochemistry|Protein|Ac...,Emmanuel Orta-Zavalza|Marcela Briones-Martin...
3,e1077,RNA Isolation and Northern Blot Analysis,The northern blot is a technique used in molec...,Vero cells (kidney epithelial cells extracted ...,RNA extraction\n\t\t\n\n\t\t\t\tCells were see...,"100 mm cell culture dishes (Corning, catalog n...",,Microbiology|Microbial genetics|RNA|RNA extrac...,Ying Liao|To Sing Fung|Mei Huang|Shouguo Fang|...
4,e1090,Flow Cytometric Analysis of Autophagic Activit...,Flow cytometry allows very sensitive and relia...,"Cells lines of interest (HepG2, HUH7, CMK, K56...",Maintain cells under standard tissue culture c...,"37 °C, 5% CO2 humidified incubator|Centrifuge|...",,Microbiology|Antimicrobial assay|Autophagy ass...,Metodi Stankov|Diana Panayotova-Dimitrova|Ma...


We can already see that none of the first 4 protocols have a value for the "background" field. we will analyse the data more in depth in the following sections.

## Cleaning and feature engineering

In [9]:
df.describe()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors
count,100,100,100,100,100,100,100,100,100
unique,100,100,100,99,100,100,38,99,95
top,e68,Preparation and Immunofluorescence Staining of...,Minimal inhibition concentration (MIC) is the ...,,Making CD-UPRT expressing stable cells\n\t\t\n...,50 ml polystyrene culture tubes (sterile)|Spec...,,Immunology|Immune cell isolation|Lymphocyte|Ce...,Peichuan Zhang
freq,1,1,1,2,1,1,63,2,2


From the output above we can see that there is a unique id, title, abstract, procedure and equipment for each different protocol. Other fields have some values which are not unique. This could mean either that they have the same value or that they are empty. We will check later on whether they are empty or not.

For now, we will start by joining all the steps of each procedure into a new column which will be called '_full\_procedure\_cleaned_':

In [12]:
import re

def join_procedure_steps(procedure):
    return ' '.join(procedure.split('|'))

def clean(procedure):
    merged_procedure = join_procedure_steps(procedure)
    return re.sub('\s+', ' ', merged_procedure).strip()
    

df['full_text'] = df['title'] + '. ' + df['abstract'] + '. ' + df['procedure'] + '. ' + df['background']
df['full_text_cleaned'] = df['full_text'].apply(lambda x: clean(x))
df['full_text_cleaned'].loc[0][:500]

'Scratch Wound Healing Assay. The scratch wound healing assay has been widely adapted and modified to study the effects of a variety of experimental conditions, for instance, gene knockdown or chemical exposure, on mammalian cell migration and proliferation. In a typical scratch wound healing assay, a “wound gap” in a cell monolayer is created by scratching, and the “healing” of this gap by cell migration and growth towards the center of the gap is monitored and often quantitated. Factors that al'

We will finally add another column with the number of characters of each procedure:

In [13]:
df['num_chars_text'] = df['full_text_cleaned'].apply(lambda x: len(x))

## Initial exploration

As we have seen before, some of the values for the background, materials, categories and authors columns are not unique. We will se now whether they contain empty values or not:

In [14]:
df.head()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors,full_text,full_text_cleaned,num_chars_text
0,e100,Scratch Wound Healing Assay,The scratch wound healing assay has been widel...,"Human MDA-MB-231 cell line (ATCC, catalog numb...",Grow cells in DMEM supplemented with 10% FBS.|...,BD Falcon 24-well tissue culture plate (Fisher...,,Cancer Biology|Invasion & metastasis|Cell biol...,Yanling Chen,Scratch Wound Healing Assay. The scratch wound...,Scratch Wound Healing Assay. The scratch wound...,2583
1,e1029,ADCC Assay Protocol,Antibody-dependent cell-mediated cytotoxicity ...,Raji cells (ATCC)|A/California/04/2009 (H1N1) ...,Preperation of Target Cells\n\n\t\t\n\n\n\t\t\...,Round bottomed 96-well plate|Temperature contr...,,Immunology|Immune cell function|Cytotoxicity|C...,Vikram Srivastava|Zheng Yang|Ivan Fan Ngai...,ADCC Assay Protocol. Antibody-dependent cell-m...,ADCC Assay Protocol. Antibody-dependent cell-m...,3824
2,e1072,Catalase Activity Assay in Candida glabrata,Commensal and pathogenic fungi are exposed to ...,Yeast strains \nNote: BG14 was used as the C. ...,Preparation of total soluble extracts\n\t\t\n\...,Orbital incubator shaker|Microfuge tubes|50 ml...,,Microbiology|Microbial biochemistry|Protein|Ac...,Emmanuel Orta-Zavalza|Marcela Briones-Martin...,Catalase Activity Assay in Candida glabrata. C...,Catalase Activity Assay in Candida glabrata. C...,4207
3,e1077,RNA Isolation and Northern Blot Analysis,The northern blot is a technique used in molec...,Vero cells (kidney epithelial cells extracted ...,RNA extraction\n\t\t\n\n\t\t\t\tCells were see...,"100 mm cell culture dishes (Corning, catalog n...",,Microbiology|Microbial genetics|RNA|RNA extrac...,Ying Liao|To Sing Fung|Mei Huang|Shouguo Fang|...,RNA Isolation and Northern Blot Analysis. The ...,RNA Isolation and Northern Blot Analysis. The ...,6890
4,e1090,Flow Cytometric Analysis of Autophagic Activit...,Flow cytometry allows very sensitive and relia...,"Cells lines of interest (HepG2, HUH7, CMK, K56...",Maintain cells under standard tissue culture c...,"37 °C, 5% CO2 humidified incubator|Centrifuge|...",,Microbiology|Antimicrobial assay|Autophagy ass...,Metodi Stankov|Diana Panayotova-Dimitrova|Ma...,Flow Cytometric Analysis of Autophagic Activit...,Flow Cytometric Analysis of Autophagic Activit...,5890


In [15]:
print_empty_cols(df)

pr_id
-----
0 articles have no value for column pr_id
Int64Index([], dtype='int64')


title
-----
0 articles have no value for column title
Int64Index([], dtype='int64')


abstract
--------
0 articles have no value for column abstract
Int64Index([], dtype='int64')


materials
---------
2 articles have no value for column materials
Int64Index([46, 72], dtype='int64')


procedure
---------
0 articles have no value for column procedure
Int64Index([], dtype='int64')


equipment
---------
0 articles have no value for column equipment
Int64Index([], dtype='int64')


background
----------
63 articles have no value for column background
Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 35, 36,
            37, 39, 40, 41, 42, 43, 47, 48, 49, 50, 51, 52, 58, 84, 85, 86, 87,
            88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64')


categories
----------
0 articles h

c:\users\alex\.envs\edma\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


As we can see, for the materials and background columns this is the case (all non-unique values are empty ones). However, there are repeated values for the equipment and authors fields, which is expected.

If we are going to use the bakground column in future steps we have to notice that most of the protocols don't have a value for it.

We are going to see how the length of each procedure is distributed amongst the dataset:

In [16]:
df['num_chars_text'].describe()

count      100.000000
mean      8244.480000
std       9229.461309
min        850.000000
25%       3299.500000
50%       5916.000000
75%       9234.500000
max      71639.000000
Name: num_chars_text, dtype: float64

In [17]:
HIST_COLUMN = 'num_chars_text'
HIST_TITLE = "Procedure length distribution for the Bio-Protocols dataset"
HIST_XLABEL = "Procedure length (# of characters)"
HIST_YLABEL = "Number of protocols"

hist.load_plot(df, HIST_COLUMN, HIST_TITLE,
               HIST_XLABEL, HIST_YLABEL, True)

In [18]:
df.loc[0].full_text_cleaned

'Scratch Wound Healing Assay. The scratch wound healing assay has been widely adapted and modified to study the effects of a variety of experimental conditions, for instance, gene knockdown or chemical exposure, on mammalian cell migration and proliferation. In a typical scratch wound healing assay, a “wound gap” in a cell monolayer is created by scratching, and the “healing” of this gap by cell migration and growth towards the center of the gap is monitored and often quantitated. Factors that alter the motility and/or growth of the cells can lead to increased or decreased rate of “healing” of the gap (Lampugnani, 1999). This assay is simple, inexpensive, and experimental conditions can be easily adjusted for different purposes. The assay can also be used for a high-throughput screen platform if an automated system is used (Yarrow and Perlman, 2004).. Grow cells in DMEM supplemented with 10% FBS. Seed cells into 24-well tissue culture plate at a density that after 24 h of growth, they 

From the cells above we can see that the mean length of each procedure is about 5800 characters, and most of the protocols have procedures of 2000-7000 characters.

In [19]:
hist.save_plot(os.path.join(NOTEBOOK_2_RESULTS_DIR, '1_Protocol_procedure_length.svg'))

There was an error exporting the plot. Please verify that both Selenium and Geckodriver are installed: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.


## Saving the dataframe

Finally, we are going to save the dataframe so we can use it later on in the following steps:

In [20]:
DF_FILE_PATH = os.path.join(NOTEBOOK_2_RESULTS_DIR, 'protocols_dataframe.pkl')

df.to_pickle(DF_FILE_PATH)